<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_08_API_RESTful.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Atenção:**

Este código funciona apenas no VSCode no Ubuntu nas máquinas do LEC.

# API RESTful

**Representational State Transfer (REST)**: Transferência Representacional de Estado

Uma **API RESTful** (também conhecida como API REST) é uma interface de programação de aplicativos (API ou API Web) que está em conformidade com as restrições do estilo de arquitetura REST e permite a interação com serviços Web RESTful.

Uma **API** é um conjunto de definições e protocolos para construir e integrar software de aplicação. Às vezes, é chamado de contrato entre um provedor de informações e um usuário de informações – estabelecendo o conteúdo exigido do consumidor (a chamada) e o conteúdo exigido pelo produtor (a resposta). Por exemplo, o design da API para um serviço meteorológico pode especificar que o usuário forneça um CEP e que o produtor responda com uma resposta em duas partes, sendo a primeira a temperatura alta e a segunda a baixa.  

Em outras palavras, se você deseja interagir com um computador ou sistema para recuperar informações ou executar uma função, uma API o ajuda a comunicar o que deseja a esse sistema para que ele possa entender e atender à solicitação.

Você pode pensar em uma API como um mediador entre os usuários ou clientes e os recursos ou serviços da Web que eles desejam obter. É também uma forma de uma organização compartilhar recursos e informações enquanto mantém a segurança, o controle e a autenticação — determinando quem tem acesso a quê.

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f1/Rest-API.png" width="500px">

# Acessórios

Bibliotecas para Python:
- fastapi: bibliteca para criar uma API Rest
- nest-asyncio: permite a execução assíncrona de threads no colab
- pyngrok: tunelamento de serviço web em url válida
- uvicorn: servidor web para subir o FastAPI

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

## Banco de Dados Local SQLite3

Baixar o banco de dados do repositório:
- https://github.com/jpwhite3/northwind-SQLite3

O banco possui a estrutura abaixo:
- https://github.com/jpwhite3/northwind-SQLite3/blob/main/README.md


In [ ]:
!wget https://raw.githubusercontent.com/jpwhite3/northwind-SQLite3/master/dist/northwind.db -O northwind.db

In [ ]:
import sqlite3
import pandas as pd
conexao = sqlite3.connect('northwind.db')
pd.read_sql('SELECT name, sql  FROM sqlite_master WHERE type="table"',conexao)

# Aplicação REST API

Cria a aplicação principal do FastAPI (app)

In [ ]:
from fastapi import FastAPI # cria o serviço de Rest API

from fastapi.responses import JSONResponse # converte valores para JSON
# cria a aplicação rest api
app = FastAPI()

# domínio inicial da aplicação
@app.get('/')
def home():
  return f'Restfull API da ECAA07 Banco de Dados'

## Conectando o banco de dados

### Tabela Produtos sem Filtro

Criando a rota ```<url>/produtos``` para acessar a lista de produtos.

Exemplo:
- https://50e4-34-91-157-85.ngrok-free.app/produtos

In [ ]:
@app.get('/produtos')
def prod():
  conexao = sqlite3.connect('northwind.db')
  cursor = conexao.cursor()
  cursor.execute("SELECT * FROM Products")
  items = cursor.fetchall()
  conexao.close()
  return items

### Tabela Categories com Filtro

Visualizando Tabela

In [ ]:
conexao = sqlite3.connect('northwind.db')
pd.read_sql('SELECT CategoryID, CategoryName, Description FROM Categories',conexao)

Criando a rota ```<url>/categoria/{item_id}``` para acessar uma categoria específica do banco de dados.

Exemplos:
- https://50e4-34-91-157-85.ngrok-free.app/categoria/2
- https://50e4-34-91-157-85.ngrok-free.app/categoria/6

In [ ]:
@app.get('/categorias/{item_id}')
def categ(item_id:int):
  conexao = sqlite3.connect('northwind.db')
  cursor = conexao.cursor()
  cursor.execute("SELECT CategoryName, Description FROM Categories WHERE CategoryID=?",(item_id,))
  items = cursor.fetchall()
  conexao.close()
  return items

# Atividades

## Rota de Fornecedores por País

Crie um rota ```<url>/fornecedor/{pais}``` para selecionar os fornecedores de um determinado país.

Exemplos:
- https://50e4-34-91-157-85.ngrok-free.app/fornecedor/USA
- https://50e4-34-91-157-85.ngrok-free.app/fornecedor/Japan

In [ ]:
conexao = sqlite3.connect('northwind.db')
pd.read_sql('SELECT DISTINCT Country FROM Suppliers',conexao)

# Ativando o Serviço e Gerando Link

Só funciona fora da UNIFEI:
- Vá em https://dashboard.ngrok.com/login
- Crie uma conta.
- Gere um token de acesso: https://dashboard.ngrok.com/get-started/your-authtoken
- Insira o token no código abaixo

In [ ]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = '21bBJ3C4LOIh31WGirKRk7rPbYj_7jr77kF6TvBoUXs95REDj'

import nest_asyncio #permite execução de servidor em loop no Colab
from pyngrok import ngrok #dá um nome de DNS real para a aplicação
import uvicorn #gestor de aplicação web

connection_string = ngrok.connect(8000).public_url

print()
print('Abra no navegador:', connection_string + '/docs')
print()
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Teste a aplicação no link gerado acima.